In [1]:
import torch

In [2]:
params=torch.load('/data/users/mayx/project/github/CenterNet/models/resnet18_64_conv4_conv2.pth',map_location='cpu')

In [3]:
state_dict=params['net']

In [4]:
print(state_dict)

OrderedDict([('conv1.weight', tensor([[[[-4.9031e-08, -3.5917e-08,  7.8672e-08],
          [-7.8640e-08, -4.2423e-08, -7.0163e-08],
          [ 5.2274e-08, -8.7531e-08, -3.7061e-08]],

         [[ 5.5713e-08, -9.7361e-08, -3.1637e-08],
          [-1.1687e-07,  9.5138e-08,  2.8471e-08],
          [ 8.0856e-08,  1.8614e-08,  8.2274e-10]],

         [[-1.0751e-08,  7.4694e-08,  4.1765e-08],
          [ 3.2644e-08,  1.1229e-08, -3.1611e-08],
          [ 9.3070e-09, -3.6813e-08, -3.3583e-08]]],


        [[[-2.9075e-02,  4.7750e-02, -8.0068e-03],
          [ 6.6991e-02,  1.5920e-01,  8.1721e-02],
          [ 3.6357e-02,  1.1533e-01,  2.8265e-02]],

         [[-1.5802e-03,  6.5645e-02,  2.0672e-02],
          [ 3.9333e-02,  1.0542e-01,  4.8184e-02],
          [ 1.4011e-02,  7.0019e-02,  2.6420e-03]],

         [[ 2.3282e-02,  7.3228e-02,  2.0897e-02],
          [ 2.5797e-02,  7.1299e-02,  1.9114e-02],
          [-4.5358e-02, -9.0126e-03, -6.2937e-02]]],


        [[[-4.0164e-02, -1.1921e-01,

In [10]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        if stride!=1:
            self.conv1=nn.Sequential(
            nn.Conv2d(in_planes, self.expansion*planes, kernel_size=4, stride=stride,padding=1, bias=False),
            nn.BatchNorm2d(self.expansion*planes)
        )
        else:
            self.conv1 = nn.Sequential(
            nn.Conv2d(in_planes, self.expansion*planes, kernel_size=3, stride=stride,padding=1, bias=False),
            nn.BatchNorm2d(self.expansion*planes)
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            if stride!=1:
                self.shortcut=nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=2, stride=stride,padding=0, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
            else:
                self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class L2Norm(nn.Module):
    def __init__(self, inplanes, gamma_init=10):
        super(L2Norm, self).__init__()
        self.gamma_init = torch.Tensor(1, inplanes, 1, 1)
        self.gamma_init[...] = gamma_init
        self.gamma_init = Parameter(self.gamma_init)

    def forward(self, x):
        x = F.normalize(x, p=2, dim=1)
        x = x * self.gamma_init
        return x
    
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(512, num_classes)
        self.norm1=L2Norm(16)
        self.norm2=L2Norm(16)
        self.norm3=L2Norm(16)
        self.norm4=L2Norm(16)
        self.trans1=nn.Conv2d(64,
                     16,
                     kernel_size=1,
                     stride=1,
                     padding=0,
                     bias=False)
        self.trans2=nn.ConvTranspose2d(in_channels=128,
                                        out_channels=16,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1,
                                        output_padding=0,
                                        bias=False)
        self.trans3=nn.ConvTranspose2d(in_channels=256,
                                        out_channels=16,
                                        kernel_size=4,
                                        stride=4,
                                        padding=0,
                                        output_padding=0,
                                        bias=False)
        self.trans4=nn.ConvTranspose2d(512,
                     16,
                     kernel_size=4,
                     stride=4,
                     padding=0,
                     bias=False)
        self.s_conv=nn.Conv2d(64,
                     16,
                     kernel_size=3,
                     stride=1,
                     padding=1,
                     bias=False)
        self.hm=nn.Conv2d(16,
             1,
             kernel_size=1,
             stride=1,
             padding=0,)
        self.wh=nn.Conv2d(16,
             2,
             kernel_size=1,
             stride=1,
             padding=0,)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        s1 = self.layer1(x)
        s2 = self.layer2(s1)
        s3 = self.layer3(s2)
        s4 = self.layer4(s3)
        s1=self.norm1(self.trans1(s1))
        s2=self.norm2(self.trans2(s2))
        s3=self.norm3(self.trans3(s3))
        s4=self.trans4(s4)
        s4=F.interpolate(s4,
                        scale_factor=2,
                        mode='bilinear')
        s4=self.norm4(s4)
        s=torch.cat([s1,s2,s3,s4],1)
        s=self.s_conv(s)
        hm_small=self.hm(s)
        wh_small=self.wh(s)
        return dict(hm_small=hm_small,wh_small=wh_small)

def ResNet18(**kwargs):
    return ResNet(BasicBlock, [2,2,2,2,2],**kwargs)

def ResNet34(**kwargs):
    return ResNet(BasicBlock, [3,4,6,3],**kwargs)

def ResNet50(**kwargs):
    return ResNet(Bottleneck, [3,4,6,3],**kwargs)

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

# test()


In [12]:
import math
model=ResNet18()
model_dict=model.state_dict()
model_dict.update(state_dict)
model.load_state_dict(model_dict)
for k,v in model.named_parameters():
    v.requires_grad=False
    if 'layer3' in k:
        break
for k, v in model.named_parameters():
    if 'hm' in k:
        if 'bias' in k:
            v = torch.ones_like(v) * -math.log(
                (1 - 0.01) / 0.01)
for k,v in model.named_parameters():
    print(k,v.requires_grad)
for k, v in model.named_parameters():
    if 'hm' in k:
        if 'bias' in k:
            print(k,v)
#             v = torch.ones_like(v) * -math.log(
#                 (1 - 0.01) / 0.01)

conv1.weight False
bn1.weight False
bn1.bias False
layer1.0.conv1.0.weight False
layer1.0.conv1.1.weight False
layer1.0.conv1.1.bias False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.1.conv1.0.weight False
layer1.1.conv1.1.weight False
layer1.1.conv1.1.bias False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer2.0.conv1.0.weight False
layer2.0.conv1.1.weight False
layer2.0.conv1.1.bias False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2.bias False
layer2.0.shortcut.0.weight False
layer2.0.shortcut.1.weight False
layer2.0.shortcut.1.bias False
layer2.1.conv1.0.weight False
layer2.1.conv1.1.weight False
layer2.1.conv1.1.bias False
layer2.1.bn1.weight False
layer2.1.bn1.bias False
layer2.1.conv2.weight False
layer2.1.bn2.weight False
layer2.1.bn2

In [14]:
a=torch.ones(10,3,704,704).cuda(1)
model.cuda(1)
output=model(a)
out=output['hm_small'].mean()+output['wh_small'].mean()
out.backward()